<a href="https://colab.research.google.com/github/niltontac/EspAnalise-EngDados/blob/master/Bolsonaro_tweets_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisando tópicos nos tweets do Presidente Jair Bolsonaro de agosto/2017 a agosto/2018.

```

https://raw.githubusercontent.com/ufrpe-ensino/curso-mineracao-textos/master/data/bolsonaro.csv

```

##Importando dados

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/ufrpe-ensino/curso-mineracao-textos/master/data/bolsonaro.csv'
df = pd.read_csv(url, encoding='utf-8')
df.head()

,data,tweet
0,2017-08-01,venezuela esta sendo destruida pelo socialismo...
1,2017-08-01,quem pariu poa voz do pt paulo henrique amorim...
2,2017-08-01,toddynho pro moleque
3,2017-08-01,a apagou mas temos o
4,2017-08-01,falamos de agronegocio e n de grafeno e no mai...


##Consultando e limpando os dados

In [ ]:
df.shape

(1741, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1741 entries, 0 to 1740
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    1741 non-null   object
 1   tweet   1734 non-null   object
dtypes: object(2)
memory usage: 27.3+ KB


In [ ]:
df.isna().sum()

data     0
tweet    7
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

data     0
tweet    0
dtype: int64

In [ ]:
df

,data,tweet
0,2017-08-01,venezuela esta sendo destruida pelo socialismo...
1,2017-08-01,quem pariu poa voz do pt paulo henrique amorim...
2,2017-08-01,toddynho pro moleque
3,2017-08-01,a apagou mas temos o
4,2017-08-01,falamos de agronegocio e n de grafeno e no mai...
...,...,...
1736,2018-08-16,os rumos da educacao no brasil precisam mudar ...
1737,2018-08-16,o psdb mais que nunca se unindo ao a narrativa...
1738,2018-08-16,o jogo de caas que a imprensa persegue mente e...
1739,2018-08-16,conhecam a verdadeira wal de mambucaba angra dos


##Scikit-learn

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV

vectorizer = CountVectorizer(analyzer='word',
                             min_df=10,                       #Número mínimo de ocorrências da palavra
                             lowercase=True,                  #Convertendo para minúsculos
                             token_pattern='[a-zA-Z0-9]{4,}', #Palavras com pelo menos 3 caracteres
                             )

tokens_cv = vectorizer.fit_transform(df['tweet'])
tokens_cv

<1734x232 sparse matrix of type '<class 'numpy.int64'>'
	with 5967 stored elements in Compressed Sparse Row format>

In [ ]:
#Construíndo um modelo LDA

lda_model = LatentDirichletAllocation(n_components=10,          #Quantidade de tópicos
                                      max_iter=300,              #Quantidade máxima de iterações 
                                      learning_method='online',
                                      random_state=100,
                                      batch_size=128,  
                                      evaluate_every = -1,
                                      n_jobs = -1                #Quantidade de CPU's
                                      )

lda_output = lda_model.fit_transform(tokens_cv)

In [ ]:
#Função para Visualizar tópicos

import numpy as np

def visualizar_topicos(vectorizer=vectorizer, lda_model='', n_words=20):
  keywords = np.array(vectorizer.get_feature_names())
  topic_keywords = []
  for topic_weights in lda_model.components_:
    top_keyword_locs = (-topic_weights).argsort()[:n_words]
    topic_keywords.append(keywords.take(top_keyword_locs))
  return topic_keywords

topic_keywords = visualizar_topicos(vectorizer=vectorizer, lda_model=lda_model, n_words=15)

#Tópicos - Palavras chaves do Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Palavra '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Tópico '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Palavra 0,Palavra 1,Palavra 2,Palavra 3,Palavra 4,Palavra 5,Palavra 6,Palavra 7,Palavra 8,Palavra 9,Palavra 10,Palavra 11,Palavra 12,Palavra 13,Palavra 14
Tópico 0,bolsonaro,jair,imprensa,voce,lula,canal,eleicoes,esse,youtube,assista,direita,ponto,unidos,link,fosse
Tópico 1,isso,sera,midia,corrupcao,temos,nome,qualquer,apos,controle,contra,nesta,lado,democracia,segundo,vitoria
Tópico 2,mesmo,sempre,seus,governo,cada,deputado,revista,este,psdb,antes,campanha,temer,ninguem,logo,direitos
Tópico 3,para,esta,brasil,mais,agora,vamos,sempre,tambem,todo,fazer,pouco,melhor,eles,nossas,nada
Tópico 4,obrigado,muito,nossa,pelo,todos,abraco,verdade,parabens,deus,apoio,voces,agradeco,minha,chegada,fakenews
Tópico 5,sobre,pela,todos,quem,contra,hoje,mais,consideracao,estamos,grato,noite,pessoas,video,onde,valeu
Tópico 6,como,brasil,mais,amor,presidente,globo,muitos,mudar,nunca,menos,somente,educacao,mulheres,favor,defesa
Tópico 7,nosso,pode,paulo,quando,porque,povo,pesquisa,futuro,trabalho,brasil,anos,folha,estado,pais,general
Tópico 8,esquerda,para,entrevista,tudo,fala,sistema,voto,essa,suas,quer,jair,semana,sendo,ainda,enquanto
Tópico 9,grande,abraco,poder,amigos,entre,plano,alguns,estao,dizer,problema,policia,alem,sociedade,camara,blica


## Preprocessamento

Tokenização utilizando o TweetTokenizer do NLTK.

In [ ]:
from nltk.tokenize import TweetTokenizer 

tweet_tokenizer = TweetTokenizer(strip_handles=False,
                                 preserve_case=False)
tweet_tokenizer.tokenize(df.iloc[90]['tweet'])

['jair', 'bolsonaro', 'e', 'entrevistado', 'pelo', 'em', 'ribeirao', 'preto']

In [ ]:
import re
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer(strip_handles=True,
                    reduce_len=True,
                    preserve_case=False)

def preprocessamento(text, join=True):

  #removendo links, colocando em minúsculos, removendo vírgulas, pontos, hífens e dois pontos
  text = re.sub(r'http\S+', '', text).lower().replace(',', '').replace('.', '').replace('-', '').replace(':', '')
  if join:
    text = ' '.join(tt.tokenize(text))
  else:
    text = tt.tokenize(text)
  return text

tokens_nltk = vectorizer.fit_transform(df['tweet'].apply(preprocessamento))
tokens_nltk

<1734x232 sparse matrix of type '<class 'numpy.int64'>'
	with 5967 stored elements in Compressed Sparse Row format>

In [ ]:
# Modelo LDA
lda_model = LatentDirichletAllocation(n_components=10,            # Número de tópicos
                                      max_iter=300,               # Número de interações
                                      learning_method='online',
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1,       
                                      n_jobs = -1,               # Número de CPUs
                                     )

lda_output = lda_model.fit_transform(tokens_nltk)

topic_keywords = visualizar_topicos(vectorizer=vectorizer, lda_model=lda_model, n_words=15)

#Tópicos - Palavras chaves do Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Palavra '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Tópico '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Palavra 0,Palavra 1,Palavra 2,Palavra 3,Palavra 4,Palavra 5,Palavra 6,Palavra 7,Palavra 8,Palavra 9,Palavra 10,Palavra 11,Palavra 12,Palavra 13,Palavra 14
Tópico 0,bolsonaro,jair,imprensa,voce,lula,canal,eleicoes,esse,youtube,assista,direita,ponto,unidos,link,fosse
Tópico 1,isso,sera,midia,corrupcao,temos,nome,qualquer,apos,controle,contra,nesta,lado,democracia,segundo,vitoria
Tópico 2,mesmo,sempre,seus,governo,cada,deputado,revista,este,psdb,antes,campanha,temer,ninguem,logo,direitos
Tópico 3,para,esta,brasil,mais,agora,vamos,sempre,tambem,todo,fazer,pouco,melhor,eles,nossas,nada
Tópico 4,obrigado,muito,nossa,pelo,todos,abraco,verdade,parabens,deus,apoio,voces,agradeco,minha,chegada,fakenews
Tópico 5,sobre,pela,todos,quem,contra,hoje,mais,consideracao,estamos,grato,noite,pessoas,video,onde,valeu
Tópico 6,como,brasil,mais,amor,presidente,globo,muitos,mudar,nunca,menos,somente,educacao,mulheres,favor,defesa
Tópico 7,nosso,pode,paulo,quando,porque,povo,pesquisa,futuro,trabalho,brasil,anos,folha,estado,pais,general
Tópico 8,esquerda,para,entrevista,tudo,fala,sistema,voto,essa,suas,quer,jair,semana,sendo,ainda,enquanto
Tópico 9,grande,abraco,poder,amigos,entre,plano,alguns,estao,dizer,problema,policia,alem,sociedade,camara,blica


## Visualizando tópicos com o [pyLDAvis](https://pypi.org/project/pyLDAvis/)

In [ ]:
!pip install pyLDAvis

In [ ]:
# Construíndo dashboard interativo para visualizar os principais tópicos identificados pelo pyLDAvis

# Mapa de distância intertópica
# Lista com os 30 principais tópicos

import pyLDAvis.sklearn
import pyLDAvis
import matplotlib.pyplot as plt

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model,
                                 tokens_nltk,
                                 vectorizer,
                                 mds='tsne',
                                 sort_topics=False)
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  10.806414
1       0.648955  71.707100       2        1   7.249886
2      71.019684   9.971006       3        1   6.472465
3       3.785981  33.185135       4        1  14.981613
4     -42.666500  -9.222070       5        1  10.704962
5      -3.621689 -43.526516       6        1  12.786249
6      -3.632402  -4.171708       7        1   9.702399
7     -36.584400  35.804413       8        1   9.813883
8      44.391380  50.615383       9        1  10.582388
9      40.235809 -31.591665      10        1   6.899740, topic_info=          Term        Freq       Total Category  logprob  loglift
24   bolsonaro  183.000000  183.000000  Default  30.0000  30.0000
152       para  276.000000  276.000000  Default  29.0000  29.0000
146   obrigado  108.000000  108.000000  Default  28.0000  28.0000
41        como   98.000000   98.000000  Default  27.0000  27.0000
104     grande   73.000000   73.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
37        caso    3.536627   24.995051  Topic10  -4.7872   0.7182
58   desespero    0.105753    9.721602  Topic10  -8.2970  -1.8473
66    ditadura    0.105771    9.866479  Topic10  -8.2968  -1.8619
53       crime    0.105791   11.503121  Topic10  -8.2966  -2.0152
49    corrupto    0.105787   11.398153  Topic10  -8.2967  -2.0061

[358 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         5  0.399631   abraco
0        10  0.588346   abraco
1         4  0.943802  adiante
2         5  0.961048  aeropoo
3         4  0.792841    agora
...     ...       ...      ...
227       2  0.909354  vitoria
228       1  0.963103     voce
229       5  0.945473    voces
230       9  0.966021     voto
231       1  0.971552  youtube

[327 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

### Salvando pyLDAvis como HTML

In [ ]:
pyLDAvis.save_html(panel, 'LDA.html')

## Visualizando tópicos com [Gensim](https://radimrehurek.com/gensim/)

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import Phrases

docs = df['tweet'].apply(lambda x: preprocessamento(x, join=False)).values

#Adicionando bigrams e trigrams to docs (apenas aqueles que aparecem 20 vezes ou mais)
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
  for token in bigram[docs[idx]]:
    if '_' in token:
      # Token é um bigram, adicione ao documento
      docs[idx].append(token)

# Criando uma representação de dicionário dos documentos
dictionary = Dictionary(docs)

# Filtrando palavras que ocorrem em menos de 20 documentos ou em mais que 50% dos documentos
dictionary.filter_extremes(no_below=20, no_above=0.5)

# Representação de Bag-of-words dos documentos
corpus = [dictionary.doc2bow(doc) for doc in docs] # Convertendo cada doc para um vetor usando BOW

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Number of unique tokens: 157
Number of documents: 1734


In [ ]:
# Treinando modelo LDA para o Gensim

from gensim.models import LdaModel

# Definindo parametros de treinamento
num_topics = 10
chunksize = 3000
passes = 30
iterations = 500
eval_every = None # Avaliar a perplexidade do modelo é muito custoso

# Construíndo um index para o dicionário de palavras
temp = dictionary[0] # Carregando o dicionário
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
model.show_topics()

[(0,
  '0.153*"bolsonaro" + 0.106*"e" + 0.103*"jair" + 0.092*"de" + 0.061*"o" + 0.048*"jair_bolsonaro" + 0.047*"a" + 0.035*"sobre" + 0.030*"fala" + 0.029*"em"'),
 (1,
  '0.112*"um" + 0.092*"obrigado" + 0.072*"abraco" + 0.061*"a" + 0.047*"de" + 0.045*"e" + 0.036*"todos" + 0.033*"pela" + 0.030*"um_abraco" + 0.028*"muito"'),
 (2,
  '0.083*"o" + 0.077*"que" + 0.066*"e" + 0.059*"de" + 0.053*"a" + 0.045*"para" + 0.037*"um" + 0.033*"nao" + 0.032*"do" + 0.028*"se"'),
 (3,
  '0.141*"de" + 0.104*"a" + 0.062*"que" + 0.062*"e" + 0.036*"do" + 0.034*"em" + 0.032*"o" + 0.026*"as" + 0.026*"sua" + 0.022*"por"'),
 (4,
  '0.128*"o" + 0.083*"que" + 0.073*"e" + 0.054*"em" + 0.047*"a" + 0.041*"nao" + 0.040*"da" + 0.034*"de" + 0.021*"isso" + 0.019*"mas"'),
 (5,
  '0.136*"e" + 0.124*"a" + 0.053*"nao" + 0.051*"o" + 0.043*"de" + 0.037*"para" + 0.028*"que" + 0.027*"da" + 0.020*"mas" + 0.020*"se"'),
 (6,
  '0.100*"e" + 0.097*"de" + 0.068*"sao" + 0.060*"o" + 0.049*"do" + 0.037*"que" + 0.035*"a" + 0.034*"por" + 0.0

In [ ]:
# Construíndo dashboard interativo para visualizar os principais tópicos identificados pelo Gensim

# Mapa de distância intertópica
# Lista com os 30 principais tópicos

import pyLDAvis.gensim
panel = pyLDAvis.gensim.prepare(model, corpus, dictionary)
panel

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.053581  0.064746       1        1  16.485195
3      0.006813  0.011074       2        1  12.210092
4      0.075393  0.066527       3        1  12.108719
5      0.048802  0.035130       4        1  11.653649
9     -0.023769 -0.100621       5        1  11.246418
8     -0.039332  0.006669       6        1   8.496738
1     -0.255108 -0.001302       7        1   8.116388
6      0.047632 -0.012029       8        1   7.361108
0      0.077494 -0.167639       9        1   7.072242
7      0.008495  0.097445      10        1   5.249458, topic_info=          Term        Freq       Total Category  logprob  loglift
40   bolsonaro  191.000000  191.000000  Default  30.0000  30.0000
92        jair  107.000000  107.000000  Default  29.0000  29.0000
3           do  522.000000  522.000000  Default  28.0000  28.0000
102   obrigado  108.000000  108.000000  Default  27.0000  27.0000
25          um  312.000000  312.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
46          em   15.902174  374.732513  Topic10  -3.8634  -0.2127
28         com   11.445168  309.768829  Topic10  -4.1922  -0.3512
45          da   10.984048  401.708008  Topic10  -4.2334  -0.6522
11          no    7.492401  297.070404  Topic10  -4.6159  -0.7330
25          um    6.118863  312.813049  Topic10  -4.8184  -0.9872

[440 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
15        1  0.132840        a
15        2  0.193989        a
15        3  0.087506        a
15        4  0.219292        a
15        5  0.076963        a
...     ...       ...      ...
35        1  0.484148     voto
35        2  0.069164     voto
35        8  0.034582     voto
35       10  0.345820     voto
154       6  0.932165  youtube

[804 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 6, 10, 9, 2, 7, 1, 8])